*Read data form bronze layer*

In [0]:
%sql
SELECT * FROM bara_slaes_project.bronze.crm_prd_info;

In [0]:
df=spark.table("bara_slaes_project.bronze.crm_prd_info")

---
*Data Transformation*

In [0]:
# *Rename Cols*
rename_mapping = {
    'prd_id':'Product_ID',
    'prd_key':'Product_Key',
    'prd_nm':'Product_name',
    'prd_cost':'Product_Cost',
    'prd_line':'Product_Line',
    'prd_start_dt':'Product_Start_Date',
    'prd_end_dt':'Product_End_Date'
}

#df = df.withColumnsRenamed(rename_mapping)
for old_name,new_name in rename_mapping.items():
    df = df.withColumnRenamed(old_name, new_name)

In [0]:
## Trim
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import *
for field in df.schema.fields:
    if isinstance(field.dataType,StringType):
        df=df.withColumn(field.name,trim(col(field.name)))

In [0]:
# *Replace Null Values*
median_Product_Cost = df.approxQuantile("Product_Cost", [0.5], 0.01)[0]
Replae_na_values={
    "Product_Cost":median_Product_Cost,
    "Product_Line":"Unknown"}
df=df.fillna(value=Replae_na_values)

*For Diplicated Product_Key, we notices that the Uplicated is due to the validation of the Product,meaning if the product End date is Null --> the Product is VALID, otherwise its not , so as a result we will conside the valid products only-Of course after getting Business Owner approval-*

In [0]:
# remove duplicates form the primary key (Customer_ID)
from pyspark.sql import functions as F
from pyspark.sql.window import Window
window_spec_last =Window.partitionBy('Product_Key').orderBy(F.asc('Product_End_Date'))
df_ranked_last=df.withColumn('row_num', F.row_number().over(window_spec_last))
#df_ranked_last.filter(df_ranked_last.row_num == 1).display()#.drop("row_num")
df=df_ranked_last.filter(df_ranked_last.row_num == 1).drop("row_num")

In [0]:
# Re-form the Product Key as it is the primary key
from pyspark.sql.functions import regexp_replace

df=(
    df
    .withColumn("Category_ID",substring("Product_Key", 1, 5))
    .withColumn("Category_ID",regexp_replace(col("Category_ID"),"-","_"))
    .withColumn("Product_Key", expr("substring(Product_Key, 7)"))
    
)


In [0]:
# *Rename Values*
Product_Line_replacements = {
    "M":"Mountain",
    "R": "Road",
    "S": "Other Sales",
    "T": "Touring",
}
df=df.replace(Product_Line_replacements,subset=["Product_Line"])

---
*Write into Silver layer*

In [0]:
df.write.mode("overwrite")\
    .format("delta")\
        .saveAsTable("bara_slaes_project.silver.product_info")

In [0]:
%sql
SELECT * FROM bara_slaes_project.silver.product_info;